In [82]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Date    : 2018-01-18 20:16:39
# @Author  : guanglinzhou (xdzgl812@163.com)

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from collections import Counter, defaultdict
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import time
project_path = '/Users/guanglinzhou/Desktop/DC_Mac/DC_Code/'
train_data = pd.read_csv(project_path + 'preprocess/traindata_output/traindata_output.csv')
test_data = pd.read_csv(project_path + 'preprocess/testdata_output/testdata_output.csv')


In [83]:
print('开始训练...')
params = {
#         'tree_method':'gpu_hist',
        'learning_rate' : 0.01,
        'n_estimators': 150,
        'max_depth': 8,
        'min_child_weight': 3,
        'gamma': 1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'silent': 1,
        'eval_metric':'auc',
        'objective':'binary:logistic',
        'scale_pos_weight':0.5
        }
print('开始CV 10折训练...')
scores = []
t0 = time.time()
train_preds = np.zeros(train_data.shape[0])
test_preds = np.zeros((test_data.shape[0], 10))
kf = KFold(n_splits=10, shuffle=True, random_state=1015)
k = kf.split(train_data)
predictors = [f for f in test_data.columns if f not in ['orderType']]
# print(predictors)

开始训练...
开始CV 10折训练...


In [ ]:
for i, (train_index, test_index) in enumerate(k):
    print('第{}次训练...'.format(i))
#     print(train_index)
#     print(test_index)
    train_feat1 = train_data.iloc[train_index]
    train_feat2 = train_data.iloc[test_index]
    xgb_train1 = xgb.DMatrix(train_feat1[predictors], label=train_feat1['orderType'])#, categorical_feature=['性别'])
    xgb_train2 = xgb.DMatrix(train_feat2[predictors], label=train_feat2['orderType'])#, categorical_feature=['性别'])
    gbm = xgb.train(params,
                    xgb_train1,
                    num_boost_round=10000,
                    evals=[(xgb_train1,'train_auc'),(xgb_train2,'test_auc')],
                    verbose_eval=100,
                    early_stopping_rounds=120,
                    )
    watchlist = [(xgb_train1,'train_auc'),(xgb_train2,'test_auc')]
    print('start training...')
#     gbm = xgb.train(params, xgb_train1, num_boost_round=800, evals=watchlist, early_stopping_rounds=40)
    feat_imp = pd.Series(gbm.get_fscore(), index=predictors).sort_values(ascending=False)
    train_preds[test_index] += gbm.predict( xgb.DMatrix(train_feat2[predictors]))
    test_preds[:, i] = gbm.predict(xgb.DMatrix(test_data[predictors]))
print('线下得分：    {}'.format(roc_auc_score(train_data['orderType'], train_preds)))
print('CV训练用时{}秒'.format(time.time() - t0))

第0次训练...
[0]	train_auc-auc:0.897055	test_auc-auc:0.885881
Multiple eval metrics have been passed: 'test_auc-auc' will be used for early stopping.

Will train until test_auc-auc hasn't improved in 120 rounds.
[100]	train_auc-auc:0.946212	test_auc-auc:0.933824
[200]	train_auc-auc:0.954962	test_auc-auc:0.940663
[300]	train_auc-auc:0.963641	test_auc-auc:0.948088
[400]	train_auc-auc:0.97091	test_auc-auc:0.952734
[500]	train_auc-auc:0.975122	test_auc-auc:0.955677
[600]	train_auc-auc:0.978224	test_auc-auc:0.957924
[700]	train_auc-auc:0.980676	test_auc-auc:0.959249
[800]	train_auc-auc:0.98265	test_auc-auc:0.960321
[900]	train_auc-auc:0.98433	test_auc-auc:0.961075
[1000]	train_auc-auc:0.985895	test_auc-auc:0.961876
[1100]	train_auc-auc:0.987202	test_auc-auc:0.962577
[1200]	train_auc-auc:0.988308	test_auc-auc:0.962928
[1300]	train_auc-auc:0.98936	test_auc-auc:0.96322
[1400]	train_auc-auc:0.990316	test_auc-auc:0.963445
[1500]	train_auc-auc:0.991183	test_auc-auc:0.963709
[1600]	train_auc-auc:0.991

[100]	train_auc-auc:0.9471	test_auc-auc:0.930823
[200]	train_auc-auc:0.955623	test_auc-auc:0.938002
[300]	train_auc-auc:0.964292	test_auc-auc:0.943415
[400]	train_auc-auc:0.97114	test_auc-auc:0.949438
[500]	train_auc-auc:0.975267	test_auc-auc:0.953388
[600]	train_auc-auc:0.978127	test_auc-auc:0.955764
[700]	train_auc-auc:0.980565	test_auc-auc:0.957397
[800]	train_auc-auc:0.982597	test_auc-auc:0.958408
[900]	train_auc-auc:0.984387	test_auc-auc:0.959116
[1000]	train_auc-auc:0.985774	test_auc-auc:0.959683
[1100]	train_auc-auc:0.987137	test_auc-auc:0.960099
[1200]	train_auc-auc:0.988299	test_auc-auc:0.960546
[1300]	train_auc-auc:0.98933	test_auc-auc:0.960693
[1400]	train_auc-auc:0.990266	test_auc-auc:0.960839
[1500]	train_auc-auc:0.99109	test_auc-auc:0.96102
[1600]	train_auc-auc:0.991881	test_auc-auc:0.961041
[1700]	train_auc-auc:0.992619	test_auc-auc:0.961087
[1800]	train_auc-auc:0.993246	test_auc-auc:0.961077
Stopping. Best iteration:
[1692]	train_auc-auc:0.99257	test_auc-auc:0.961134

s

In [ ]:
print('线下得分：    {}'.format(roc_auc_score(train_data['orderType'], train_preds)))
print('CV训练用时{}秒'.format(time.time() - t0))

In [74]:
test_preds.shape

(10076, 10)

In [75]:
pred=test_preds.mean(axis=1)

In [76]:
pred

array([ 0.02527279,  0.00226583,  0.00414515, ...,  0.26153967,
        0.00200982,  0.66667446])

In [77]:
result=pd.DataFrame(columns=['userid','orderType'])

In [78]:
result['userid']=test_data.userid

In [79]:
result['orderType']=pred

In [80]:
result

,userid,orderType
0,100000000371,0.025273
1,100000001215,0.002266
2,100000001445,0.004145
3,100000001575,0.997767
4,100000001601,0.000333
5,100000001737,0.999601
6,100000001829,0.004136
7,100000002055,0.001107
8,100000002241,0.004738
9,100000003067,0.001061


In [81]:
result.to_csv('/Users/guanglinzhou/Desktop/result.csv',index=False)